In [12]:
!pip install mediapipe opencv-python


In [13]:

import cv2
import mediapipe as mp
import time

In [14]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Create the pose object
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)


In [15]:
# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Ignoring empty frame.")
        continue

    # Convert the BGR image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Process the image and detect pose
    results = pose.process(image)

    # Draw the pose annotation on the image
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Show the final output
    cv2.imshow('Real-Time Pose Detection', image)

    # Exit with 'q' key
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [16]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Start webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally (so you see yourself like a mirror)
    frame = cv2.flip(frame, 1)

    # Prepare two copies of the frame
    original = frame.copy()                     # Left side: original
    skeleton = np.zeros_like(frame)            # Right side: black background

    # Convert to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb_frame.flags.writeable = False

    # Detect pose
    results = pose.process(rgb_frame)

    # If landmarks detected, draw them on the black skeleton image
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            skeleton,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(255,0,0), thickness=2)
        )

    # Combine original and skeleton side-by-side
    combined = np.hstack((original, skeleton))

    # Show the result
    cv2.imshow('You (Left) vs Skeleton (Right)', combined)

    # Exit with 'q'
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
ca


NameError: name 'ca' is not defined

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Start webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally (so you see yourself like a mirror)
    frame = cv2.flip(frame, 1)

    # Prepare two copies of the frame
    original = frame.copy()                     # Left side: original
    skeleton = np.zeros_like(frame)            # Right side: black background

    # Convert to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb_frame.flags.writeable = False

    # Detect pose
    results = pose.process(rgb_frame)

    # If landmarks detected, draw them on the black skeleton image
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            skeleton,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(255,0,0), thickness=2)
        )

    # Combine original and skeleton side-by-side
    combined = np.hstack((original, skeleton))

    # Show the result
    cv2.imshow('You (Left) vs Skeleton (Right)', combined)

    # Exit with 'q'
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# ✅ Corrected: release the camera
cap.release()
cv2.destroyAllWindows()


In [ ]:
import socket
import cv2
import mediapipe as mp
import json

# Initialize MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
cap = cv2.VideoCapture(0)

# Create a socket
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # UDP socket
server_address = ('127.0.0.1', 5050)  # Send to Unity on localhost

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    data = {}

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        key_indices = [0, 11, 12, 13, 14, 23, 24]
        for idx in key_indices:
            lm = landmarks[idx]
            data[str(idx)] = {'x': lm.x, 'y': lm.y, 'z': lm.z}

        # Convert to JSON and send
        message = json.dumps(data).encode()
        sock.sendto(message, server_address)

    cv2.imshow("Pose", frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
sock.close()


In [ ]:
import socket
import cv2
import mediapipe as mp
import json

# Initialize MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Open webcam
cap = cv2.VideoCapture(0)

# Create UDP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('127.0.0.1', 5050)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    data = {}

    if results.pose_landmarks:
        print("Pose detected ✅")
        landmarks = results.pose_landmarks.landmark
        key_indices = [0, 11, 12, 13, 14, 23, 24]  # Selected joints

        for idx in key_indices:
            lm = landmarks[idx]
            data[str(idx)] = {'x': lm.x, 'y': lm.y, 'z': lm.z}
            print(f"Joint {idx}: x={lm.x:.2f}, y={lm.y:.2f}, z={lm.z:.2f}")

        # Send JSON data over socket
        message = json.dumps(data).encode()
        sock.sendto(message, server_address)
        print("Data sent to Unity 🚀")

    else:
        print("❌ Pose not detected. Please stand in camera view.")

    # Show the webcam frame
    cv2.imshow("Webcam", frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
sock.close()


In [ ]:
import socket
import cv2
import mediapipe as mp
import json
import numpy as np

# Initialize MediaPipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Open webcam
cap = cv2.VideoCapture(0)

# Create UDP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('127.0.0.1', 5050)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip frame to mirror image
    frame = cv2.flip(frame, 1)

    # Original feed for the left side
    original = frame.copy()

    # Black image for the skeleton side
    skeleton = np.zeros_like(frame)

    # Convert to RGB
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    data = {}

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        key_indices = [0, 11, 12, 13, 14, 23, 24]  # Nose, shoulders, elbows, hips

        for idx in key_indices:
            lm = landmarks[idx]
            data[str(idx)] = {'x': lm.x, 'y': lm.y, 'z': lm.z}
            print(f"Joint {idx}: x={lm.x:.2f}, y={lm.y:.2f}, z={lm.z:.2f}")

        # Send JSON to Unity
        message = json.dumps(data).encode()
        sock.sendto(message, server_address)
        print("Data sent to Unity 🚀")

        # Draw pose landmarks on the black image
        mp_drawing.draw_landmarks(
            skeleton,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
        )
    else:
        print("❌ Pose not detected")

    # Combine both views side-by-side
    combined = np.hstack((original, skeleton))

    # Show the result
    cv2.imshow("You (Left) vs Skeleton (Right)", combined)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
sock.close()


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Load the anime face image
face_img = cv2.imread('anime_face.png', cv2.IMREAD_UNCHANGED)
if face_img is None:
    print("❌ Couldn't load face image. Make sure 'anime_face.png' is in the same folder.")
    raise SystemExit

# Resize face image
face_img = cv2.resize(face_img, (100, 100))

# Init MediaPipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Webcam capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    # Left view = original webcam
    left = frame.copy()

    # Right view = black canvas
    right = np.zeros_like(frame)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Draw stickman on right
        mp_drawing.draw_landmarks(
            right,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=4, circle_radius=4),
            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=4)
        )

        # Get head (nose) landmark = index 0
        nose = landmarks[0]
        h, w, _ = right.shape
        cx, cy = int(nose.x * w), int(nose.y * h)

        # Overlay anime face at nose location
        face_h, face_w, _ = face_img.shape

        # Top-left corner of face image
        x1 = cx - face_w // 2
        y1 = cy - face_h // 2
        x2 = x1 + face_w
        y2 = y1 + face_h

        # Ensure bounds are within image size
        if x1 >= 0 and y1 >= 0 and x2 <= w and y2 <= h:
            roi = right[y1:y2, x1:x2]
            face_rgb = face_img[:, :, :3]
            face_alpha = face_img[:, :, 3] / 255.0

            for c in range(3):  # RGB channels
                roi[:, :, c] = roi[:, :, c] * (1 - face_alpha) + face_rgb[:, :, c] * face_alpha

            right[y1:y2, x1:x2] = roi

    combined = np.hstack((left, right))
    cv2.imshow("You (Left) | Anime Twin (Right)", combined)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import socket
import cv2
import mediapipe as mp
import json
import numpy as np

# Setup MediaPipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Setup UDP socket for Unity
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('127.0.0.1', 5050)

# Start Webcam
cap = cv2.VideoCapture(0)

def draw_cartoon_avatar(landmarks, img, width, height):
    points = {}
    for idx in [0, 11, 12, 13, 14, 23, 24]:  # Nose, shoulders, elbows, hips
        lm = landmarks[idx]
        x, y = int(lm.x * width), int(lm.y * height)
        points[idx] = (x, y)

    # Cartoon head
    if 0 in points:
        cv2.circle(img, points[0], 30, (255, 204, 102), -1)  # Face skin color
        cv2.circle(img, points[0], 5, (0, 0, 0), -1)         # Nose

    # Cartoon body (shoulders to hips)
    if 11 in points and 12 in points:
        cv2.line(img, points[11], points[12], (0, 255, 0), 4)
    if 11 in points and 23 in points:
        cv2.line(img, points[11], points[23], (0, 255, 0), 4)
    if 12 in points and 24 in points:
        cv2.line(img, points[12], points[24], (0, 255, 0), 4)

    # Cartoon arms
    if 11 in points and 13 in points:
        cv2.line(img, points[11], points[13], (255, 0, 0), 4)
    if 12 in points and 14 in points:
        cv2.line(img, points[12], points[14], (255, 0, 0), 4)

    return img

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    original = frame.copy()
    h, w = frame.shape[:2]
    cartoon = np.full_like(frame, 255)  # White background

    # Convert to RGB
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    data = {}

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        key_indices = [0, 11, 12, 13, 14, 23, 24]  # Select major joints

        for idx in key_indices:
            lm = landmarks[idx]
            data[str(idx)] = {'x': lm.x, 'y': lm.y, 'z': lm.z}

        # Send data to Unity
        message = json.dumps(data).encode()
        sock.sendto(message, server_address)
        print("✅ Sent to Unity")

        # Draw cartoon avatar
        cartoon = draw_cartoon_avatar(landmarks, cartoon, w, h)
    else:
        print("❌ Pose not detected")

    # Combine original and cartoon
    combined = np.hstack((original, cartoon))
    cv2.imshow("You (Left) vs Cartoon Avatar (Right)", combined)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
sock.close()


In [ ]:
pose = mp_pose.Pose(model_complexity=1, enable_segmentation=False)


In [ ]:
data = {}

for idx, lm in enumerate(results.pose_landmarks.landmark):
    data[str(idx)] = {'x': lm.x, 'y': lm.y, 'z': lm.z}

# Send full body pose
message = json.dumps(data).encode()
sock.sendto(message, server_address)


In [ ]:
import socket
import cv2
import mediapipe as mp
import json
import numpy as np

# MediaPipe setup
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(model_complexity=1)

# UDP setup
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('127.0.0.1', 5050)


In [ ]:
def draw_mesh_body(landmarks, canvas, w, h):
    pts = [(int(lm.x * w), int(lm.y * h)) for lm in landmarks]
    # Draw filled mesh polygons between pose connections
    for connection in mp_pose.POSE_CONNECTIONS:
        p1, p2 = connection
        if p1 < len(pts) and p2 < len(pts):
            cv2.line(canvas, pts[p1], pts[p2], (30, 144, 255), 4)
    # Optionally fill torso
    torso = [pts[i] for i in [11,12,23,24]]
    cv2.fillPoly(canvas, [np.array(torso, dtype=np.int32)], (32, 178, 170))
    return canvas


In [ ]:
cap = cv2.VideoCapture(0)
print("Press 'q' to quit")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    h, w = frame.shape[:2]
    canvas = np.zeros_like(frame)

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Send all pose points
        data = {str(i): {'x': lm.x, 'y': lm.y, 'z': lm.z}
                for i, lm in enumerate(landmarks)}
        sock.sendto(json.dumps(data).encode(), server_address)

        # Draw mesh avatar
        canvas = draw_mesh_body(landmarks, canvas, w, h)
    else:
        cv2.putText(canvas, "No pose detected", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    combined = np.hstack((frame, canvas))
    cv2.imshow("You (Left) vs Mesh Avatar (Right)", combined)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
sock.close()


In [ ]:
import socket
import cv2
import mediapipe as mp
import json
import numpy as np

# MediaPipe setup
mp_pose = mp.solutions.pose
mp_face = mp.solutions.face_mesh

pose = mp_pose.Pose(model_complexity=1)
face_mesh = mp_face.FaceMesh(static_image_mode=False, max_num_faces=1)

# UDP setup
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('127.0.0.1', 5050)


In [ ]:
def draw_mesh_body(landmarks, canvas, w, h):
    pts = [(int(lm.x * w), int(lm.y * h)) for lm in landmarks]
    for connection in mp_pose.POSE_CONNECTIONS:
        p1, p2 = connection
        if p1 < len(pts) and p2 < len(pts):
            cv2.line(canvas, pts[p1], pts[p2], (30, 144, 255), 4)
    torso = [pts[i] for i in [11, 12, 24, 23] if i < len(pts)]
    if len(torso) == 4:
        cv2.fillPoly(canvas, [np.array(torso)], (32, 178, 170))
    return canvas

def detect_face_emoji(face_landmarks, w, h):
    # Get upper lip and lower lip points
    top_lip = face_landmarks.landmark[13]
    bottom_lip = face_landmarks.landmark[14]
    lip_dist = abs(bottom_lip.y - top_lip.y)

    # Define emoji based on mouth openness
    if lip_dist > 0.05:
        return "😮"  # Mouth open
    else:
        return "😊"  # Neutral/smile
